In [1]:
import glob
import re

# import cv2
import pandas as pd
# import matplotlib.pyplot as plt

In [ ]:
# download hart16 files from: https://data.galaxyzoo.org/#section-12
# download images and mapping from: https://zenodo.org/record/3565489#.Y3vFKS-l0eY
FILENAME_MAPPINGS = "data/gz2_filename_mapping.csv"
HART16 = "data/gz2_hart16.csv"

ORIGINAL_IMAGES_DIR = "data/images/"
PROCESSED_IMAGES_DIR = "data/images_processed/"

## Process CSV Files

In [3]:
# dtypes for Hart columns
# note, objid column must use Int64, int is too small
dtype_hart = {
    "dr7objid": "Int64",
    "ra": float,
    "dec": float,
    "rastring": str,
    "decstring": str,
    "sample": str,
    "gz2_class": str,
    "total_classifications": int,
    "total_votes": int,
    "t01_smooth_or_features_a01_smooth_count": int,
    "t01_smooth_or_features_a01_smooth_weight": float,
    "t01_smooth_or_features_a01_smooth_fraction": float,
    "t01_smooth_or_features_a01_smooth_weighted_fraction": float,
    "t01_smooth_or_features_a01_smooth_debiased": float,
    "t01_smooth_or_features_a01_smooth_flag": int,
    "t01_smooth_or_features_a02_features_or_disk_count": int,
    "t01_smooth_or_features_a02_features_or_disk_weight": float,
    "t01_smooth_or_features_a02_features_or_disk_fraction": float,
    "t01_smooth_or_features_a02_features_or_disk_weighted_fraction": float,
    "t01_smooth_or_features_a02_features_or_disk_debiased": float,
    "t01_smooth_or_features_a02_features_or_disk_flag": int,
    "t01_smooth_or_features_a03_star_or_artifact_count": int,
    "t01_smooth_or_features_a03_star_or_artifact_weight": float,
    "t01_smooth_or_features_a03_star_or_artifact_fraction": float,
    "t01_smooth_or_features_a03_star_or_artifact_weighted_fraction": float,
    "t01_smooth_or_features_a03_star_or_artifact_debiased": float,
    "t01_smooth_or_features_a03_star_or_artifact_flag": int,
    "t02_edgeon_a04_yes_count": int,
    "t02_edgeon_a04_yes_weight": float,
    "t02_edgeon_a04_yes_fraction": float,
    "t02_edgeon_a04_yes_weighted_fraction": float,
    "t02_edgeon_a04_yes_debiased": float,
    "t02_edgeon_a04_yes_flag": int,
    "t02_edgeon_a05_no_count": int,
    "t02_edgeon_a05_no_weight": float,
    "t02_edgeon_a05_no_fraction": float,
    "t02_edgeon_a05_no_weighted_fraction": float,
    "t02_edgeon_a05_no_debiased": float,
    "t02_edgeon_a05_no_flag": int,
    "t03_bar_a06_bar_count": int,
    "t03_bar_a06_bar_weight": float,
    "t03_bar_a06_bar_fraction": float,
    "t03_bar_a06_bar_weighted_fraction": float,
    "t03_bar_a06_bar_debiased": float,
    "t03_bar_a06_bar_flag": int,
    "t03_bar_a07_no_bar_count": int,
    "t03_bar_a07_no_bar_weight": float,
    "t03_bar_a07_no_bar_fraction": float,
    "t03_bar_a07_no_bar_weighted_fraction": float,
    "t03_bar_a07_no_bar_debiased": float,
    "t03_bar_a07_no_bar_flag": int,
    "t04_spiral_a08_spiral_count": int,
    "t04_spiral_a08_spiral_weight": float,
    "t04_spiral_a08_spiral_fraction": float,
    "t04_spiral_a08_spiral_weighted_fraction": float,
    "t04_spiral_a08_spiral_debiased": float,
    "t04_spiral_a08_spiral_flag": int,
    "t04_spiral_a09_no_spiral_count": int,
    "t04_spiral_a09_no_spiral_weight": float,
    "t04_spiral_a09_no_spiral_fraction": float,
    "t04_spiral_a09_no_spiral_weighted_fraction": float,
    "t04_spiral_a09_no_spiral_debiased": float,
    "t04_spiral_a09_no_spiral_flag": int,
    "t05_bulge_prominence_a10_no_bulge_count": int,
    "t05_bulge_prominence_a10_no_bulge_weight": float,
    "t05_bulge_prominence_a10_no_bulge_fraction": float,
    "t05_bulge_prominence_a10_no_bulge_weighted_fraction": float,
    "t05_bulge_prominence_a10_no_bulge_debiased": float,
    "t05_bulge_prominence_a10_no_bulge_flag": int,
    "t05_bulge_prominence_a11_just_noticeable_count": int,
    "t05_bulge_prominence_a11_just_noticeable_weight": float,
    "t05_bulge_prominence_a11_just_noticeable_fraction": float,
    "t05_bulge_prominence_a11_just_noticeable_weighted_fraction": float,
    "t05_bulge_prominence_a11_just_noticeable_debiased": float,
    "t05_bulge_prominence_a11_just_noticeable_flag": int,
    "t05_bulge_prominence_a12_obvious_count": int,
    "t05_bulge_prominence_a12_obvious_weight": float,
    "t05_bulge_prominence_a12_obvious_fraction": float,
    "t05_bulge_prominence_a12_obvious_weighted_fraction": float,
    "t05_bulge_prominence_a12_obvious_debiased": float,
    "t05_bulge_prominence_a12_obvious_flag": int,
    "t05_bulge_prominence_a13_dominant_count": int,
    "t05_bulge_prominence_a13_dominant_weight": float,
    "t05_bulge_prominence_a13_dominant_fraction": float,
    "t05_bulge_prominence_a13_dominant_weighted_fraction": float,
    "t05_bulge_prominence_a13_dominant_debiased": float,
    "t05_bulge_prominence_a13_dominant_flag": int,
    "t06_odd_a14_yes_count": int,
    "t06_odd_a14_yes_weight": float,
    "t06_odd_a14_yes_fraction": float,
    "t06_odd_a14_yes_weighted_fraction": float,
    "t06_odd_a14_yes_debiased": float,
    "t06_odd_a14_yes_flag": int,
    "t06_odd_a15_no_count": int,
    "t06_odd_a15_no_weight": float,
    "t06_odd_a15_no_fraction": float,
    "t06_odd_a15_no_weighted_fraction": float,
    "t06_odd_a15_no_debiased": float,
    "t06_odd_a15_no_flag": int,
    "t07_rounded_a16_completely_round_count": int,
    "t07_rounded_a16_completely_round_weight": float,
    "t07_rounded_a16_completely_round_fraction": float,
    "t07_rounded_a16_completely_round_weighted_fraction": float,
    "t07_rounded_a16_completely_round_debiased": float,
    "t07_rounded_a16_completely_round_flag": int,
    "t07_rounded_a17_in_between_count": int,
    "t07_rounded_a17_in_between_weight": float,
    "t07_rounded_a17_in_between_fraction": float,
    "t07_rounded_a17_in_between_weighted_fraction": float,
    "t07_rounded_a17_in_between_debiased": float,
    "t07_rounded_a17_in_between_flag": int,
    "t07_rounded_a18_cigar_shaped_count": int,
    "t07_rounded_a18_cigar_shaped_weight": float,
    "t07_rounded_a18_cigar_shaped_fraction": float,
    "t07_rounded_a18_cigar_shaped_weighted_fraction": float,
    "t07_rounded_a18_cigar_shaped_debiased": float,
    "t07_rounded_a18_cigar_shaped_flag": int,
    "t08_odd_feature_a19_ring_count": int,
    "t08_odd_feature_a19_ring_weight": float,
    "t08_odd_feature_a19_ring_fraction": float,
    "t08_odd_feature_a19_ring_weighted_fraction": float,
    "t08_odd_feature_a19_ring_debiased": float,
    "t08_odd_feature_a19_ring_flag": int,
    "t08_odd_feature_a20_lens_or_arc_count": int,
    "t08_odd_feature_a20_lens_or_arc_weight": float,
    "t08_odd_feature_a20_lens_or_arc_fraction": float,
    "t08_odd_feature_a20_lens_or_arc_weighted_fraction": float,
    "t08_odd_feature_a20_lens_or_arc_debiased": float,
    "t08_odd_feature_a20_lens_or_arc_flag": int,
    "t08_odd_feature_a21_disturbed_count": int,
    "t08_odd_feature_a21_disturbed_weight": float,
    "t08_odd_feature_a21_disturbed_fraction": float,
    "t08_odd_feature_a21_disturbed_weighted_fraction": float,
    "t08_odd_feature_a21_disturbed_debiased": float,
    "t08_odd_feature_a21_disturbed_flag": int,
    "t08_odd_feature_a22_irregular_count": int,
    "t08_odd_feature_a22_irregular_weight": float,
    "t08_odd_feature_a22_irregular_fraction": float,
    "t08_odd_feature_a22_irregular_weighted_fraction": float,
    "t08_odd_feature_a22_irregular_debiased": float,
    "t08_odd_feature_a22_irregular_flag": int,
    "t08_odd_feature_a23_other_count": int,
    "t08_odd_feature_a23_other_weight": float,
    "t08_odd_feature_a23_other_fraction": float,
    "t08_odd_feature_a23_other_weighted_fraction": float,
    "t08_odd_feature_a23_other_debiased": float,
    "t08_odd_feature_a23_other_flag": int,
    "t08_odd_feature_a24_merger_count": int,
    "t08_odd_feature_a24_merger_weight": float,
    "t08_odd_feature_a24_merger_fraction": float,
    "t08_odd_feature_a24_merger_weighted_fraction": float,
    "t08_odd_feature_a24_merger_debiased": float,
    "t08_odd_feature_a24_merger_flag": int,
    "t08_odd_feature_a38_dust_lane_count": int,
    "t08_odd_feature_a38_dust_lane_weight": float,
    "t08_odd_feature_a38_dust_lane_fraction": float,
    "t08_odd_feature_a38_dust_lane_weighted_fraction": float,
    "t08_odd_feature_a38_dust_lane_debiased": float,
    "t08_odd_feature_a38_dust_lane_flag": int,
    "t09_bulge_shape_a25_rounded_count": int,
    "t09_bulge_shape_a25_rounded_weight": float,
    "t09_bulge_shape_a25_rounded_fraction": float,
    "t09_bulge_shape_a25_rounded_weighted_fraction": float,
    "t09_bulge_shape_a25_rounded_debiased": float,
    "t09_bulge_shape_a25_rounded_flag": int,
    "t09_bulge_shape_a26_boxy_count": int,
    "t09_bulge_shape_a26_boxy_weight": float,
    "t09_bulge_shape_a26_boxy_fraction": float,
    "t09_bulge_shape_a26_boxy_weighted_fraction": float,
    "t09_bulge_shape_a26_boxy_debiased": float,
    "t09_bulge_shape_a26_boxy_flag": int,
    "t09_bulge_shape_a27_no_bulge_count": int,
    "t09_bulge_shape_a27_no_bulge_weight": float,
    "t09_bulge_shape_a27_no_bulge_fraction": float,
    "t09_bulge_shape_a27_no_bulge_weighted_fraction": float,
    "t09_bulge_shape_a27_no_bulge_debiased": float,
    "t09_bulge_shape_a27_no_bulge_flag": int,
    "t10_arms_winding_a28_tight_count": int,
    "t10_arms_winding_a28_tight_weight": float,
    "t10_arms_winding_a28_tight_fraction": float,
    "t10_arms_winding_a28_tight_weighted_fraction": float,
    "t10_arms_winding_a28_tight_debiased": float,
    "t10_arms_winding_a28_tight_flag": int,
    "t10_arms_winding_a29_medium_count": int,
    "t10_arms_winding_a29_medium_weight": float,
    "t10_arms_winding_a29_medium_fraction": float,
    "t10_arms_winding_a29_medium_weighted_fraction": float,
    "t10_arms_winding_a29_medium_debiased": float,
    "t10_arms_winding_a29_medium_flag": int,
    "t10_arms_winding_a30_loose_count": int,
    "t10_arms_winding_a30_loose_weight": float,
    "t10_arms_winding_a30_loose_fraction": float,
    "t10_arms_winding_a30_loose_weighted_fraction": float,
    "t10_arms_winding_a30_loose_debiased": float,
    "t10_arms_winding_a30_loose_flag": int,
    "t11_arms_number_a31_1_count": int,
    "t11_arms_number_a31_1_weight": float,
    "t11_arms_number_a31_1_fraction": float,
    "t11_arms_number_a31_1_weighted_fraction": float,
    "t11_arms_number_a31_1_debiased": float,
    "t11_arms_number_a31_1_flag": int,
    "t11_arms_number_a32_2_count": int,
    "t11_arms_number_a32_2_weight": float,
    "t11_arms_number_a32_2_fraction": float,
    "t11_arms_number_a32_2_weighted_fraction": float,
    "t11_arms_number_a32_2_debiased": float,
    "t11_arms_number_a32_2_flag": int,
    "t11_arms_number_a33_3_count": int,
    "t11_arms_number_a33_3_weight": float,
    "t11_arms_number_a33_3_fraction": float,
    "t11_arms_number_a33_3_weighted_fraction": float,
    "t11_arms_number_a33_3_debiased": float,
    "t11_arms_number_a33_3_flag": int,
    "t11_arms_number_a34_4_count": int,
    "t11_arms_number_a34_4_weight": float,
    "t11_arms_number_a34_4_fraction": float,
    "t11_arms_number_a34_4_weighted_fraction": float,
    "t11_arms_number_a34_4_debiased": float,
    "t11_arms_number_a34_4_flag": int,
    "t11_arms_number_a36_more_than_4_count": int,
    "t11_arms_number_a36_more_than_4_weight": float,
    "t11_arms_number_a36_more_than_4_fraction": float,
    "t11_arms_number_a36_more_than_4_weighted_fraction": float,
    "t11_arms_number_a36_more_than_4_debiased": float,
    "t11_arms_number_a36_more_than_4_flag": int,
    "t11_arms_number_a37_cant_tell_count": int,
    "t11_arms_number_a37_cant_tell_weight": float,
    "t11_arms_number_a37_cant_tell_fraction": float,
    "t11_arms_number_a37_cant_tell_weighted_fraction": float,
    "t11_arms_number_a37_cant_tell_debiased": float,
    "t11_arms_number_a37_cant_tell_flag": int
}

# columns to keep from Hart
hart_keep = ["dr7objid", "ra", "dec", "gz2_class"]
# Keep only debiased columns and the above 4 columns
for col_name in dtype_hart.keys():
    if col_name.endswith("_debiased"):
        hart_keep.append(col_name)

### Load the CSV Files

In [4]:
df_mappings = pd.read_csv(FILENAME_MAPPINGS,
                          header=0,
                          dtype={"objid": "Int64",
                                 "sample": str,
                                 "asset_id": int})
df_mappings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355990 entries, 0 to 355989
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   objid     355990 non-null  Int64 
 1   sample    355990 non-null  object
 2   asset_id  355990 non-null  int32 
dtypes: Int64(1), int32(1), object(1)
memory usage: 7.1+ MB


In [5]:
df_hart16 = pd.read_csv(HART16,
                        header=0,
                        dtype=dtype_hart,
                        usecols=hart_keep)
df_hart16.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239695 entries, 0 to 239694
Data columns (total 41 columns):
 #   Column                                                Non-Null Count   Dtype  
---  ------                                                --------------   -----  
 0   dr7objid                                              239695 non-null  Int64  
 1   ra                                                    239695 non-null  float64
 2   dec                                                   239695 non-null  float64
 3   gz2_class                                             239695 non-null  object 
 4   t01_smooth_or_features_a01_smooth_debiased            239695 non-null  float64
 5   t01_smooth_or_features_a02_features_or_disk_debiased  239695 non-null  float64
 6   t01_smooth_or_features_a03_star_or_artifact_debiased  239695 non-null  float64
 7   t02_edgeon_a04_yes_debiased                           239695 non-null  float64
 8   t02_edgeon_a05_no_debiased                  

### Check for duplicates

In [6]:
print("Duplicates in objID to image mapping file")
df_mappings.duplicated("objid").value_counts()

Duplicates in objID to image mapping file


False    325651
True      30339
dtype: int64

In [7]:
print("Duplicates in Hart16 data file")
df_hart16.duplicated("dr7objid").value_counts()

Duplicates in Hart16 data file


False    239695
dtype: int64

In [8]:
_dupe_df_mappings = df_mappings[df_mappings.duplicated("objid", keep=False)]\
                                .sort_values("asset_id")\
                                .reset_index(drop=True)
_dupe_id_first = _dupe_df_mappings['asset_id'].iloc[0]
_dupe_id_last = _dupe_df_mappings['asset_id'].iloc[-1]
print(f"asset_id in objid duplicates: First: {_dupe_id_first}, Last: {_dupe_id_last}, diff:{_dupe_id_last-_dupe_id_first}, count:{_dupe_df_mappings.shape[0]}")
print("None of these are in the provided images")

asset_id in objid duplicates: First: 295306, Last: 355990, diff:60684, count:60678
None of these are in the provided images


In [9]:
# drop rows with asset_id not in image list to remove duplicates
df_mappings_clean = df_mappings[df_mappings["asset_id"] < 295306]
print("Are there duplicates in mappings now?")
df_mappings_clean.duplicated("objid").value_counts()

Are there duplicates in mappings now?


False    295305
dtype: int64

### Merge the two DataFames

In [10]:
df_joined = pd.merge(df_mappings_clean, df_hart16,
                     left_on="objid",
                     right_on="dr7objid",
                     how="inner",
                     validate="one_to_one",
                     sort=False)
print(df_joined.shape)
df_joined.head()

(239695, 44)


,objid,sample,asset_id,dr7objid,ra,dec,gz2_class,t01_smooth_or_features_a01_smooth_debiased,t01_smooth_or_features_a02_features_or_disk_debiased,t01_smooth_or_features_a03_star_or_artifact_debiased,...,t09_bulge_shape_a27_no_bulge_debiased,t10_arms_winding_a28_tight_debiased,t10_arms_winding_a29_medium_debiased,t10_arms_winding_a30_loose_debiased,t11_arms_number_a31_1_debiased,t11_arms_number_a32_2_debiased,t11_arms_number_a33_3_debiased,t11_arms_number_a34_4_debiased,t11_arms_number_a36_more_than_4_debiased,t11_arms_number_a37_cant_tell_debiased
0,587722981741363294,original,3,587722981741363294,182.925262,-1.092357,Sb,0.459,0.681,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,587722981741363323,original,4,587722981741363323,182.970108,-1.219537,Sc?l,0.216,0.736,0.000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,587722981741559888,original,5,587722981741559888,183.438095,-1.238414,Er,0.773,0.029,0.114,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,587722981741625481,original,6,587722981741625481,183.473999,-1.231429,Sc1t,0.152,0.639,0.009,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,587722981741625484,original,7,587722981741625484,183.477783,-1.084604,Sb,0.182,0.720,0.032,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Check Images
See if images in folder match up to `asset_id` in the DataFrame

In [11]:
# get list of all images in the image folder
image_files = glob.glob(ORIGINAL_IMAGES_DIR + "*.jpg")
print("Image count:", len(image_files))

# Get image names, these should match up to `asset_id`
regex = re.compile(r"images\\(\d*)\.jpg")
image_names = [int(re.search(regex, img).group(1)) for img in image_files]
image_names.sort()
print("First 10 image names:")
image_names[0:10]

Image count: 243434
First 10 image names:


[3, 4, 5, 6, 7, 8, 9, 11, 12, 13]

In [12]:
table_asset = df_joined["asset_id"].to_list()
# create list of images that are not in images
not_in_images = list(set(table_asset) - set(image_names))
print("Not in images, In table (drop these rows)", len(not_in_images))

# create list of data that is not in the table
not_in_table = list(set(image_names) - set(table_asset))
print("Not in table, In images (do not process these images)", len(not_in_table))

Not in images, In table (drop these rows) 122
Not in table, In images (do not process these images) 3861


Remove rows that do not have images

In [51]:
# Keep rows that are not in (`~`) `not_in_images` list
df_joined_clean = df_joined[~df_joined["asset_id"].isin(not_in_images)]
df_joined_clean.shape

(239573, 44)

Create a new column that reduces the number of `gz2_class` options. When we separate our data into testing and training, we want to stratify the data, but we can only use 1 column. The `gz2_class` column is perfect for this but there are many values that only have 1 or 2 matching rows.

We will use the cutoff at `8`

In [53]:
gz2class_vc = df_hart16["gz2_class"].value_counts()
print("Total classes:", gz2class_vc.shape[0])

very_rare_classes = gz2class_vc.loc[gz2class_vc < 8]
very_rare_classes

Total classes: 818


SBc+l       7
Sd4m(i)     7
Sc4t(o)     7
SBb3t(m)    7
SBc2t(u)    7
           ..
SBc4t(u)    1
SBc+l(m)    1
Sa?l(i)     1
SBd+m(i)    1
Sd3m(o)     1
Name: gz2_class, Length: 348, dtype: int64

In [48]:
regex_spiral_arm_and_winding = re.compile(r"([1234+?][tml])")

def convert_rare_class(original_class_name: str) -> str:
    """Removes the spiral arm count and spiral winding parts of a
    string if there is a rare feature: (x)

    Use this to reduce the amount of gz2_class strings for better stratifying

    :param original_class_name: GZ2 Class name
    :return: Class name without spiral parts
    """
    # If there is no rare feature, don't change the class
    if not original_class_name.endswith(")"):
        return original_class_name
    
    # Elliptical galaxies probably are not here anyways
    if original_class_name.startswith("E"):
        return original_class_name
    
    # Get the (x) feature
    rare_feature = original_class_name[-3:]
    
    # Check for spiral arm count and winding
    re_match = re.search(regex_spiral_arm_and_winding, original_class_name)
    if re_match is not None:
        # Get the shape, bar, bulge prominence & combine with rare feature
        shape_bar_bulge = original_class_name[:-5]
        return shape_bar_bulge + rare_feature
    else:
        # Nothing to remove, return original
        return original_class_name

# Reduce number of rare classes
very_rare_class_mapping = {}
for orig_class_name in very_rare_classes.index.to_list():
    new_name = convert_rare_class(orig_class_name)
    very_rare_class_mapping[orig_class_name] = new_name

# count of new classes
print("Original very rare classes count:", len(very_rare_classes.index.to_list()))
print("Reduced very rare classes count:", len(set(very_rare_class_mapping.values())))

Original very rare classes count: 348
Reduced very rare classes count: 95


In [56]:
# create a new column from `gz2_class` and replace the rare classes with our new classes
df_joined_clean["class_reduced"] = df_joined_clean["gz2_class"].replace(very_rare_class_mapping)
print("Total reduced classes:", df_joined_clean["class_reduced"].value_counts().shape[0])

Total reduced classes: 525


C:\Users\Zachary\AppData\Local\Temp\ipykernel_13664\2020128925.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_joined_clean["class_reduced"] = df_joined_clean["gz2_class"].replace(very_rare_class_mapping)


# Clean Images
Process for cleaning images:
- Skip if image in `not_in_table` list
- Find main central feature of image
- Crop to feature
- Convert to gray-scale
- Scale down to 106×106 (¼ original size of 424×424)
- Save to new folder with same name